In [ ]:
import requests
import json
import gradio as gr
from PIL import Image, ImageDraw, ImageFont
import platform

def request_vision(image_path):
    #endpoint, api_key 정보
    try:
        endpoint = "https://1507-computervision-we.cognitiveservices.azure.com/computervision/imageanalysis:analyze?"
        api_key = "450e09da0b934eb2bc950eb9f51fa754"
    except KeyError:
        print("Missing environment variable 'VISION_ENDPOINT' or 'VISION_KEY'")
        print("Set them before running this sample.")
        exit()

    # 헤더 생성
    headers = {
        "Content-Type": "application/octet-stream",
        "Ocp-Apim-Subscription-Key": api_key
    }

    # 파라미터 생성
    params ={
        "api-version":"2024-02-01",
        "features":"read",
        "language":"ko"
    }

    # 이미지 데이터 불러온 후 api 호출
    with open(image_path, 'rb') as image:
        image_data = image.read()

    response = requests.post(endpoint,
                             headers=headers,
                             params=params,
                             data=image_data)

    # 받은 리스폰스를 json으로 파싱 후 리턴
    response_json = response.json()
    return response_json

def change_image(image_path):
    image = Image.open(image_path)
    draw = ImageDraw.Draw(image)
    if platform.system() == 'Windows':
        font = ImageFont.truetype('malgun.ttf', size=15)
    elif platform.system() == 'Darwin':
        font = ImageFont.truetype('AppleGothic.ttf', size=15)
    else:
        font = ImageFont.load_default(size=15)

    # request_vision으로 사진 이미지 전송
    response_json = request_vision(image_path=image_path)
    if "readResult" in response_json and "blocks" in response_json['readResult']:
        block_list = response_json['readResult']['blocks']

        for block in block_list:
            line_list = block['lines']
            for line in line_list:
                text = line['text']
                bounding_polygon = line['boundingPolygon']
                polygon = list(map(lambda p: (p['x'], p['y']), bounding_polygon))

                draw.polygon(polygon, outline='red', fill=None, width=3)
                draw.text((bounding_polygon[3]['x'], bounding_polygon[3]['y'] + 3), text=text, fill='green', font=font)

    return image

with gr.Blocks() as demo:
    with gr.Row():
        input_image = gr.Image(label="이미지 선택", type="filepath", width=800)
        output_image = gr.Image(label="결과 이미지", type="pil", interactive=False, width=800)

    input_image.change(fn=change_image, inputs=[input_image], outputs=[output_image])

demo.launch()
# change_image("C:/Users/chch4/Desktop/KakaoTalk_20240626_104540950.png")
# "C:/Users/chch4/Desktop/15.png"
# print(json.dumps(request_vision("C:/Users/chch4/Desktop/15.png"), indent=2))